Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.

Example:

Input: "babad"

Output: "bab"

Note: "aba" is also a valid answer.
 

Example:

Input: "cbbd"

Output: "bb"

# Approach 1 (longest common string)

In [63]:
def longestPalindrome(s):
    '''
    :type s: str
    :rtype: str
    '''
    dim = len(s)
    rev = s[::-1]
    m = [[0 for x in range(dim+1)] for x in range(dim + 1)]
    mmax = 0
    p = 0
    for i in range(dim):
        for j in range(dim):
            if s[i] == rev[j]:
                m[i+1][j+1] = m[i][j] + 1
                if m[i+1][j+1] > mmax:
                    p = i + 1
                    mmax = m[i+1][j+1]

    for c in m:
        print(c)
    print('Max is ', max(m, axis=1))
    return s[p-mmax:p], mmax, p
print(longestPalindrome('aba'))

[0, 0, 0, 0]
[0, 1, 0, 1]
[0, 0, 2, 0]
[0, 1, 0, 3]


TypeError: 'axis' is an invalid keyword argument for this function

最长公共子串(The Longest Common Substring)

LCS问题就是求两个字符串最长公共子串的问题. 解法可以用一个矩阵来记录两个字符串中所有位置的两个字符之间的匹配情况，若是匹配则为1,否则为0。然后求出对角线最长的1的序列，其对应的位置就是最长匹配子串的位置。

In [7]:
def lcs(s1, s2):
    m = [([0]*range(len(s2)+1)) for j in range(len(s1)+1)]
    # 生成(len(s2)+1)*(len(s1)+1)的0矩阵，为方便后续计算（特别是
    # 循环的时候防止出现列下标溢出的情况），比字符串长度多了一列  
    mmax = 0 # 最长匹配的长度
    p = 0 # 最长匹配子串的最后一位在s1的序号 + 1
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i] == s2[j]:
                m[i+1][j+1] = m[i][j] + 1
                if m[i+1][j+1] > mmax:
                    mmax = m[i+1][j+1]
                    p = i + 1
    for r in m:
        print(r)
    return s1[p-mmax:p], mmax

print(lcs('abcdfg', 'abdfg'))

[0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 2, 0]
[0, 0, 0, 0, 0, 3]
('dfg', 3)


In [12]:
import numpy
def lcseq(s1, s2):
    # 生成字符串长度加1的0矩阵，m保存对应位置匹配的结果
    m = [[0 for x in range(len(s2) + 1)] for y in range(len(s1)+1)]
    # d用来记录转移方向
    d = [[None for x in range(len(s2)+1)] for y in range(len(s1)+1)]
    
    for p1 in range(len(s1)):
        for p2 in range(len(s2)):
            if s1[p1] == s2[p2]:  # 字符串匹配成功，则该位置的值为左上方加1
                m[p1+1][p2+1] = m[p1][p2] + 1
                d[p1+1][p2+1] = 'ok'
            elif m[p1+1][p2] > m[p1][p2+1]:  #左值大于上值，则该位置的值为左值，并标记回溯时的方向  
                m[p1+1][p2+1] = m[p1+1][p2]   
                d[p1+1][p2+1] = 'lf'            
            else:                           #上值大于左值，则该位置的值为上值，并标记方向up  
                m[p1+1][p2+1] = m[p1][p2+1]     
                d[p1+1][p2+1] = 'up'           
    (p1, p2) = (len(s1), len(s2))   
    print(numpy.array(d))
    s = []   
    while m[p1][p2]:    #不为None时  
        c = d[p1][p2]  
        if c == 'ok':   #匹配成功，插入该字符，并向左上角找下一个  
            s.append(s1[p1-1])  
            p1-=1  
            p2-=1   
        if c =='lf':  #根据标记，向左找下一个  
            p2 -= 1  
        if c == 'up':   #根据标记，向上找下一个  
            p1 -= 1  
    s.reverse()   
    return ''.join(s)   
  
print(lcseq('abdfg','abcdfg'))

[[None None None None None None None]
 [None 'ok' 'lf' 'lf' 'lf' 'lf' 'lf']
 [None 'up' 'ok' 'lf' 'lf' 'lf' 'lf']
 [None 'up' 'up' 'up' 'ok' 'lf' 'lf']
 [None 'up' 'up' 'up' 'up' 'ok' 'lf']
 [None 'up' 'up' 'up' 'up' 'up' 'ok']]
abdfg


# Approach II Dynamic Programming

In [102]:
# 94/94 test cases passed
# runtime: 5440 ms
def longestPalindrome(s):
    '''
    :type s: str
    :rtype: str
    :define a table to store all P(i, j), where P(i, j) = true if Si...Sj is palindrom
    and P(i, j)=false, otherwise
    The base cases are:
        P(i, i) = True
        P(i, i+1) = (Si == Si+1)
    '''
    dim = len(s)
    table = [([0]*dim) for x in range(dim)]
    for i in range(dim): 
        table[i][i] = True
        if i < dim - 1:
            table[i][i+1] = (s[i] == s[i+1])
    # First, initialize the one and two letters palindromes, and work our way up
    # finding all three letters palindromes, and so on...
    for step in range(2, dim): 
        for i in range(0, dim-step):
            table[i][i+step] = table[i+1][i+step-1] and (s[i] == s[i+step])
#     for c in table:
#         print(c)
    mmax = 0
    t = s[0]
    for col in range(dim-1,-1, -1):
        for row in range(0,col): # 对角线上最后考虑
            if table[row][col] and col+1-row > mmax: #必须引入mmax, eg.‘abadd’
                t = s[row:col+1]
                mmax = col + 1 - row
    #若没有包含至少两个字母的回文字符串，则随便输出一个字母即可
    return t, mmax
print(longestPalindrome('abadd'))

('aba', 3)


# Approach 3 Expand around enter

In [104]:
# runtime: 896 ms (in 94 tests)
def longestPalindrome(s):
    dim = len(s)
    if dim == 1:
        return s
    mmax = 0
    sbegin = 0
    for i in range(dim):
        # if dim is odd
        left, right = i - 1, i + 1
        while left >= 0 and right < dim and s[left] == s[right]:
            if mmax < right - left:
                mmax = right - left
                sbegin = left
            left -= 1
            right += 1
        
        # if dim is even
        left, right = i, i + 1
        while left >= 0 and right < dim and s[left] == s[right]:
            if mmax < right - left:
                mmax = right - left
                sbegin = left
            left -= 1
            right += 1
            
        return s[sbegin:(sbegin+mmax+1)]

print(longestPalindrome('aaaa'))

('aaaa', 4)


# Approach 4 Manacher's Algorithm

In [116]:
# runtime: 88 ms (in 94 tests)
def longestPalindrome(s):
    t = '^' + '#'+'#'.join(list(s))+'#' + '$' # 首尾加^ $是为了防止越界 (while循环)
    dim = len(t)
    # p[i] records the palindromic radius in the center of t[i]
    p = [0]* dim
    R = 0 # 已检索的回文串中能延伸到的最右端位置
    C = 0 # 对应右边是R的回文串中心
    maxlen = 0 # 最大回文串长度
    centerIndex = 0  # 最大回文串长度
    for i in range(1, dim - 1):
        '''当i<mx时，能对称取值得下标，当半径小于mx-i时，就是本身，
          大于则只能取mx-lc，超过的部分只能++的比较。当i>mx时，只能++的比较。
        '''
        i_mirror = 2 * C - i
        if R > i:
            p[i] = min(p[i_mirror], R - i)
        else:
            p[i] = 1
        # Attempt to expand palindrome centered at i
        while t[i + p[i]] == t[i - p[i]]: 
            p[i] += 1
        # If palindrome centered at i expand past R,
        # adjust center based on expanded palindrome.
        if p[i] > maxlen:
            maxlen = p[i]
            centerIndex = i
        if R < i + p[i]:
            R = i + p[i]
            C = i
    return s[(centerIndex - maxlen + 1)//2:(centerIndex + maxlen - 1)//2]        
print(longestPalindrome('abbcbae'))

bcb
